In [1]:
from pyspark.sql import SparkSession
from deep_translator import GoogleTranslator

spark = (
    
    SparkSession
            .builder
            .config('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
            .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
            .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
            .getOrCreate()
            
)

22/09/23 15:48:48 WARN Utils: Your hostname, desktop-pro resolves to a loopback address: 127.0.1.1; using 192.168.98.179 instead (on interface eth0)
22/09/23 15:48:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/carlosbarbosa/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/carlosbarbosa/.ivy2/cache
The jars for the packages stored in: /home/carlosbarbosa/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-908a4a2c-d7ca-4cc4-9efc-b456efaa10db;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 94ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| searc

In [12]:
advices = (
    spark
    .read
    .format('delta')
    .load('../datalake/bronze/advices/')
)

In [13]:
def translate_en_to_pt(text: str):
    
    return GoogleTranslator(source='english', target='portuguese').translate(text)

In [14]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [15]:
translated = F.udf(translate_en_to_pt, T.StringType())

In [16]:
advices = advices.withColumn('advice', translated('advice'))

In [17]:
advices = advices.withColumnRenamed('advice', 'conselho')

In [23]:
advices.show(n=1, vertical=True, truncate=False)

-RECORD 0--------------------------------------------------------------------------
 id       | 18                                                                     
 conselho | Não julgue um livro pela capa, a menos que tenha uma sinopse no verso. 
only showing top 1 row



In [24]:
(
    advices
    .write
    .format('delta')
    .mode('overwrite')
    .save('../datalake/silver/advices_pt')
)